In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import scipy.stats as st
from scipy.stats import skew

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from pathlib import Path

from visualization import *

In [10]:
root_dir = "/nfs2/harmonization/BIDS/WRAPnew/derivatives/"
pattern = "sub-*/ses-*/SLANT-TICVv1.2/post/FinalResult/*_T1w_seg.nii.gz"
ai_pairs_csv = "labels/slant_ai_pairs.csv"
label_index = "labels/slant_itksnap.csv"
ticv_csv = "stats_csv/ticv_summary.csv"
asymmetry_index_csv = "stats_csv/asymmetry_index.csv"
roi_volumes_csv = "stats_csv/roi_volumes.csv"
roi_volumes_zscore_csv = "stats_csv/roi_volumes_zscore.csv"

In [6]:
ticv_df = pd.read_csv(ticv_csv)
ticv_series = ticv_df["TICV_ml"].values

fig = plt.figure(constrained_layout=True, figsize=(15, 8))
gs = GridSpec(1, 2, figure=fig)
ax1 = fig.add_subplot(gs[0, 0])
parts = ax1.violinplot(ticv_series, showmeans=False, showmedians=True, widths=0.6)
for pc in parts["bodies"]:
    pc.set_alpha(0.6)
ax1.boxplot(ticv_series, widths=0.15, vert=True, showfliers=False, positions=[1])
ax1.set_ylabel("TICV (mL)")
ax1.set_xticks([])
ax1.set_title("Violin")


ax2 = fig.add_subplot(gs[0, 1])
ax2.hist(ticv_series, bins=30, alpha=0.6, density=True, label="Histogram")
kde_x = np.linspace(ticv_series.min(), ticv_series.max(), 200)
kde_y = st.gaussian_kde(ticv_series)(kde_x)
ax2.plot(kde_x, kde_y, linewidth=2, label="KDE")
ax2.axvline(ticv_series.mean(), linestyle="--", label="Mean")
ax2.set_xlabel("TICV (mL)")
ax2.set_ylabel("Density")
ax2.set_title("Histogram")
ax2.legend()
fig.suptitle("TICV Distribution QC", fontsize=14)
fig.savefig("stats_png/ticv_violin_hist.png", dpi=300)
plt.close(fig)

In [7]:
df = pd.read_csv(ticv_csv)

print(f"\nMean ± SD TICV = {df.TICV_ml.mean():.1f} ± {df.TICV_ml.std():.1f} mL")
outliers = df[df.zscore.abs() >= 2.8]
print(f"Outliers (|Z|>3) = {len(outliers)}")

print("\n--- Outlier details ---")
print(outliers[["subject", "TICV_ml", "zscore"]].to_string(index=False))

lowest = df.nsmallest(5, "TICV_ml")

print("\n--- Lowest TICV values ---")
print(lowest[["subject", "TICV_ml", "zscore"]].to_string(index=False))


Mean ± SD TICV = 191.1 ± 38.3 mL
Outliers (|Z|>3) = 18

--- Outlier details ---
                 subject    TICV_ml    zscore
 sub-wrap0387_ses-191212 492.825000  7.872702
 sub-wrap0441_ses-230902 301.604267  2.883144
 sub-wrap0455_ses-200505 348.385000  4.103802
 sub-wrap0582_ses-191108 606.870000 10.848499
 sub-wrap0583_ses-190831 302.252000  2.900045
 sub-wrap0640_ses-200213 421.131000  6.001977
 sub-wrap0654_ses-200108 454.688000  6.877586
 sub-wrap0699_ses-180411 299.961000  2.840266
 sub-wrap0699_ses-200227 451.697000  6.799541
 sub-wrap0860_ses-210203 340.987000  3.910765
 sub-wrap0912_ses-190903 434.064000  6.339440
 sub-wrap0986_ses-200310 346.060000  4.043136
sub-wrapL0013_ses-191110 331.032000  3.651007
sub-wrapL0121_ses-200109 430.266000  6.240338
sub-wrapL0138_ses-220923 305.230389  2.977761
sub-wrapL0217_ses-191104 643.157000 11.795343
sub-wrapL0224_ses-191004 492.034000  7.852062
sub-wrapL0227_ses-190627 400.351000  5.459761

--- Lowest TICV values ---
                 

In [ ]:
outliers = df[df.zscore.abs() >= 2.84]

for _, row in outliers.iterrows():
    subject = row["subject"]
    zscore = row["zscore"]
    print(f"z-score = {zscore}")
    visualize_slant_subjectid(subject)

In [9]:
vdf = pd.read_csv(roi_volumes_csv).set_index("subject").apply(pd.to_numeric)
print("mean volume range:", vdf.mean().min(), "-", vdf.mean().max(), "mL")
print("global CV:", (vdf.std() / vdf.mean()).mean())

large_roi = vdf.mean()[vdf.mean() > 150].sort_values(ascending=False)
small_roi = vdf.mean()[vdf.mean() < 0.5].sort_values()
print("Largest ROI means:\n", large_roi)
print("\nSmallest ROI means:\n", small_roi)

mean volume range: 0.3523237743099046 - 209.07497362569677 mL
global CV: 0.20527984585029585
Largest ROI means:
 L45    209.074974
L44    205.563351
dtype: float64

Smallest ROI means:
 L75    0.352324
L30    0.382686
L23    0.393496
L49    0.409389
L50    0.431639
L76    0.474865
dtype: float64


In [ ]:
def plot_roi_violin_hist(
    vdf: pd.DataFrame,
    right_id: int,
    left_id: int,
    roi_name: str | None = None,
    out_dir: str | Path = "stats_png/",
):
    out_dir = Path(out_dir)
    out_dir.mkdir(exist_ok=True)

    col_l, col_r = f"L{left_id}", f"L{right_id}"
    if col_l not in vdf.columns or col_r not in vdf.columns:
        raise KeyError(f"{col_l} or {col_r} not in DataFrame columns.")

    vol_l = vdf[col_l].dropna()
    vol_r = vdf[col_r].dropna()
    all_vals = pd.concat([vol_r.rename("Right"), vol_l.rename("Left")])

    roi_name = roi_name or f"ROI_{right_id}-{left_id}"

    fig = plt.figure(constrained_layout=True, figsize=(15, 6))
    gs = GridSpec(1, 2, figure=fig)

    # 1) Violin + Box
    ax1 = fig.add_subplot(gs[0, 0])
    parts = ax1.violinplot([vol_r, vol_l], showmedians=True, widths=0.6)
    for pc in parts["bodies"]:
        pc.set_alpha(0.6)
    ax1.boxplot(
        [vol_r, vol_l], widths=0.15, vert=True, showfliers=False, positions=[1, 2]
    )
    ax1.set_xticks([1, 2])
    ax1.set_xticklabels([f"Right ({right_id})", f"Left ({left_id})"])
    ax1.set_ylabel("Volume (mL)")
    ax1.set_title("Violin")

    # 2) Histogram + KDE
    ax2 = fig.add_subplot(gs[0, 1])
    bins = np.linspace(all_vals.min(), all_vals.max(), 30)
    ax2.hist(vol_r, bins=bins, alpha=0.6, density=True, label=f"Right {right_id}")
    ax2.hist(vol_l, bins=bins, alpha=0.6, density=True, label=f"Left {left_id}")
    for series, label in [(vol_r, "Right"), (vol_l, "Left")]:
        kde_x = np.linspace(series.min(), series.max(), 200)
        kde_y = st.gaussian_kde(series)(kde_x)
        ax2.plot(kde_x, kde_y, linewidth=2, label=f"KDE {label}")
    ax2.axvline(all_vals.mean(), ls="--", label="Mean (combined)")
    ax2.set_xlabel("Volume (mL)")
    ax2.set_ylabel("Density")
    ax2.set_title("Histogram")
    ax2.legend()

    fig.suptitle(f"{roi_name} Volume QC", fontsize=14)
    out_path = out_dir / f"{roi_name.replace(' ', '_')}_violin_hist.png"
    fig.savefig(out_path, dpi=300)
    plt.close(fig)
    print(f"Saved {out_path}")

In [15]:
def find_long_tail_roi(
    volume_df: pd.DataFrame, skew_thr: float = 1.0, fraction_thr: float = 0.05
) -> pd.DataFrame:
    rows = []
    for roi in volume_df.columns:
        series = volume_df[roi].dropna()
        s = skew(series)
        if abs(s) < skew_thr:
            continue

        m, sd = series.mean(), series.std(ddof=0)
        if s > 0:
            extreme = (series > m + 2 * sd).sum()
            frac = extreme / len(series)
            if frac >= fraction_thr:
                rows.append([roi, s, "right", frac])
        else:
            extreme = (series < m - 2 * sd).sum()
            frac = extreme / len(series)
            if frac >= fraction_thr:
                rows.append([roi, s, "left", frac])

    out_df = pd.DataFrame(rows, columns=["ROI", "skew", "tail", "extreme_frac"])
    return out_df.sort_values("skew", key=np.abs, ascending=False)

In [ ]:
vdf = pd.read_csv(roi_volumes_csv)
plot_roi_violin_hist(vdf, 51, 52)

Saved ROI_52-51_violin_hist.png


In [ ]:
roi_id = 40
thr = 3.0

zdf = pd.read_csv(roi_volumes_zscore_csv).set_index("subject").apply(pd.to_numeric)
col_name = f"L{roi_id}"
outliers = zdf.loc[zdf[col_name].abs() >= thr, col_name]

print(f"\nROI {col_name} — |z| ≥ {thr} ({len(outliers)})")
for sid, z in outliers.items():
    print(f"{sid:30s}  z = {z:+.2f}")


ROI L40 — |z| ≥ 3.0 (29)
sub-wrap0002_ses-200809         z = -3.15
sub-wrap0387_ses-191212         z = -3.41
sub-wrap0414_ses-160616         z = -5.05
sub-wrap0439_ses-150502         z = -3.48
sub-wrap0440_ses-170428         z = -4.09
sub-wrap0490_ses-170119         z = -4.27
sub-wrap0566_ses-170915         z = -4.33
sub-wrap0591_ses-170903         z = -3.80
sub-wrap0625_ses-170119         z = -5.85
sub-wrap0640_ses-200213         z = -3.11
sub-wrap0699_ses-180411         z = -5.69
sub-wrap0699_ses-200227         z = -3.17
sub-wrap0725_ses-170612         z = -3.66
sub-wrap0740_ses-191014         z = -3.74
sub-wrap0860_ses-210203         z = -5.97
sub-wrap0912_ses-190903         z = -3.32
sub-wrap0915_ses-160615         z = -4.15
sub-wrap0927_ses-170722         z = -4.46
sub-wrap0946_ses-231124         z = +4.35
sub-wrap0986_ses-200310         z = -3.22
sub-wrap1038_ses-140215         z = +3.29
sub-wrap1147_ses-220502         z = -3.86
sub-wrapL0013_ses-171111        z = -3.88
sub-wrap

In [ ]:
vdf = pd.read_csv(roi_volumes_csv).set_index("subject").apply(pd.to_numeric)
pairs_df = pd.read_csv(
    "/home-local/lij112/toolbox/slant_ai_pairs.csv",
    usecols=["RightID", "LeftID", "ROI"],
)
name_map = {
    f"L{r}": roi + "_R"
    for r, roi in pairs_df[["RightID", "ROI"]].itertuples(index=False)
}
name_map.update(
    {
        f"L{l}": roi + "_L"
        for l, roi in pairs_df[["LeftID", "ROI"]].itertuples(index=False)
    }
)
vdf = vdf.rename(columns=name_map)

tail_rois = find_long_tail_roi(vdf, skew_thr=0.5, fraction_thr=0.02)
print(tail_rois)

                                ROI      skew   tail  extreme_frac
7               Lateral-Ventricle_L  2.813958  right      0.041159
6               Lateral-Ventricle_R  2.638924  right      0.040650
8                        Pallidum_R -2.133846   left      0.024898
5               Inf-Lat-Ventricle_L  2.095165  right      0.047256
4               Inf-Lat-Ventricle_R  1.796168  right      0.046240
2           Cerebral-White-Matter_R  1.795260  right      0.026931
3           Cerebral-White-Matter_L  1.357895  right      0.024898
0         Cerebellum-White-Matter_R -0.981974   left      0.031504
14         MPoG-postcentral-gyrus_L  0.960395  right      0.041159
13         MPoG-postcentral-gyrus_R  0.843802  right      0.039126
9                 Basal-Forebrain_R -0.842146   left      0.021341
1         Cerebellum-White-Matter_L -0.783184   left      0.024898
12    AOrG-anterior-orbital-gyrus_L  0.668521  right      0.035061
11  ACgG-anterior-cingulate-gyrus_L  0.607477  right      0.03

In [ ]:
aidf = pd.read_csv(asymmetry_index_csv).set_index("subject").apply(pd.to_numeric)

pairs_df = pd.read_csv(
    ai_pairs_csv,
    usecols=["RightID", "LeftID", "ROI"],
)

map_dict = {f"AI_{r}-{l}": roi for r, l, roi in pairs_df.itertuples(index=False)}

aidf = aidf.rename(columns=map_dict)
aidf = aidf[[c for c in map_dict.values() if c in aidf.columns]]

fig, ax = plt.subplots(figsize=(14, 6))
im = ax.imshow(aidf.values, aspect="auto", cmap="coolwarm", vmin=-0.2, vmax=0.2)
plt.colorbar(im, ax=ax, label="AI")

ax.set_ylabel("Subject")
ax.set_xticks(np.arange(aidf.shape[1]))
ax.set_xticklabels(aidf.columns, rotation=90, fontsize=6)
ax.set_xlabel("ROI (Left vs. Right)")

plt.tight_layout()
plt.savefig("stats_png/ai_heatmap.png", dpi=300)
plt.close()

# ==============================================================
sub_flags = (aidf.abs() > 0.15).sum(axis=1)
sub_flags.sort_values(ascending=False).head(20).plot.bar(figsize=(9, 4))
plt.axhline(5, color="r", ls="--")
plt.ylabel("# ROI |AI|>0.15")
plt.title("Top subjects by AI outlier count")
plt.tight_layout()
plt.savefig("stats_png/ai_subject_bar.png", dpi=300)
plt.close()

# ==============================================================
top_cols = aidf.abs().mean().sort_values(ascending=False).head(10).index
sns.violinplot(
    data=aidf[top_cols].melt(var_name="ROI", value_name="AI"),
    x="ROI",
    y="AI",
    inner="box",
)
plt.axhline(0.15, color="r", ls="--")
plt.axhline(-0.15, color="r", ls="--")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig("stats_png/ai_top_roi_violin.png", dpi=300)
plt.close()

# ==============================================================
ticv = (
    pd.read_csv(ticv_csv).set_index("subject")["zscore"].reindex(aidf.index)
)
plt.scatter(ticv, aidf.abs().mean(axis=1), alpha=0.7)
plt.axhline(0.15, color="r", ls="--")
plt.axvline(2.8, color="r", ls="--")
plt.axvline(-2.8, color="r", ls="--")
plt.xlabel("TICV z-score")
plt.ylabel("Mean |AI| across ROI")
plt.title("Mean |AI| vs TICV z-score")
plt.tight_layout()
plt.savefig("stats_png/ai_vs_ticv_scatter.png", dpi=300)
plt.close()